<a href="https://colab.research.google.com/github/harin/cudsi-capstone-2019/blob/master/embedding/generate_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install spacy==2.2 spacy-transformers
!python -m spacy download en_trf_bertbaseuncased_lg

  Using cached https://files.pythonhosted.org/packages/1c/96/10881bc82d05664c40a38cf7c645e6ab1877e5808ef4a3684a99cd1445a4/spacy-2.2.0-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/4c/60/cbe7c1ca0d05e9861375fc9fe73f5c6828666bb994e193a80bce554c4004/thinc-7.1.1-cp36-cp36m-manylinux1_x86_64.whl
ERROR: spacy-transformers 0.5.1 has requirement spacy<2.3.0,>=2.2.1, but you'll have spacy 2.2.0 which is incompatible.
ERROR: en-trf-bertbaseuncased-lg 2.2.0 has requirement spacy>=2.2.1, but you'll have spacy 2.2.0 which is incompatible.
ERROR: en-core-web-sm 2.2.5 has requirement spacy>=2.2.2, but you'll have spacy 2.2.0 which is incompatible.
  Found existing installation: thinc 7.3.1
    Uninstalling thinc-7.3.1:
      Successfully uninstalled thinc-7.3.1
  Found existing installation: spacy 2.2.3
    Uninstalling spacy-2.2.3:
      Successfully uninstalled spacy-2.2.3
     |████████████████████████████████| 10.4MB 5.6MB/s 
     |███████████████████████

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import spacy
is_using_gpu = spacy.prefer_gpu()
if is_using_gpu:
    torch.set_default_tensor_type("torch.cuda.FloatTensor")

nlp = spacy.load("en_trf_bertbaseuncased_lg")
doc = nlp("Apple shares rose on the news. Apple pie is delicious.")
print(doc[0].similarity(doc[7]))
print(doc._.trf_last_hidden_state.shape)

0.5041006
(16, 768)


In [0]:
dirs(doc[0])

[]

In [0]:
doc[0].tensor

array([ 8.11704397e-01,  1.76520869e-02, -1.33570001e-01, -5.86767159e-02,
        9.99733210e-01,  1.40545905e-01,  4.13097262e-01,  1.24671984e+00,
       -1.10548191e-01, -2.55057782e-01,  3.72076154e-01, -4.47247654e-01,
       -2.29001060e-01,  6.05916858e-01, -3.85589182e-01, -3.40647519e-01,
       -2.97724724e-01,  1.05291581e+00, -1.18897177e-01, -2.27075871e-02,
       -1.20696791e-01, -6.22191727e-01,  5.84125258e-02,  1.15239337e-01,
        6.64814055e-01, -7.14252710e-01, -8.21750343e-01, -4.81725752e-01,
       -3.17274511e-01, -4.83956076e-02, -6.92197680e-02,  3.55899841e-01,
        2.59304374e-01,  2.42629424e-01, -8.21924329e-01, -4.26706403e-01,
       -6.52516365e-01,  2.80679524e-01, -8.64305437e-01,  2.74663270e-01,
       -1.55243710e-01, -9.29275572e-01,  6.57990992e-01, -4.91689071e-02,
       -1.46509290e-01, -1.52108267e-01, -1.25101686e+00,  4.68670219e-01,
        3.30374986e-01, -1.38513088e-01, -4.43823397e-01,  7.28103876e-01,
        5.84599257e-01, -

In [0]:
!ls "/content/gdrive/My Drive/DS_Capstone/data/raw/2018_07_19_04_59_08/"


articles.txt  fingerprints.txt


In [0]:
import pandas as pd
def read_article_df(file):
    articles_dt = file.read().split('\n')[:-1]
    pubId, canonicalUrl,firstScrape,title,text,lang_reliability = [],[],[],[],[],[]
    lang_iso = []
    for article in articles_dt:    
        row = article.split('\t')
        pubId.append(row[0])
        canonicalUrl.append(row[4])
        firstScrape.append(row[5])
        lang_iso.append(row[6])
        lang_reliability.append(row[7])
        title.append(row[8])
        text.append(row[9])

    articles_df = pd.DataFrame()
    articles_df['pubId'] = pubId
    articles_df['canonicalUrl'] = canonicalUrl
    articles_df['firstScrape'] = firstScrape
    articles_df['title'] = title
    articles_df['text'] = text
    articles_df['lang_reliability'] = lang_reliability
    articles_df['lang_iso'] = lang_iso
    return articles_df

In [0]:
articles = read_article_df(open("/content/gdrive/My Drive/DS_Capstone/data/raw/2018_07_19_04_59_08/articles.txt"))

In [0]:
articles.all_text = articles.title + articles.text

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


In [0]:
articles.all_text[:10]

0    "I Told You So": Trump Hits Out At EU After $5...
1    Trump Invites Putin To  Ahead Of Midterms | Ze...
2    Gallup Shows How Much Americans Really Care Ab...
3    California Supreme Court Blocks Proposal To Sp...
4    Why Are "Thousands" of Teslas Sitting In a Fie...
5    Time Magazine Goes Full Propagandist | Zero He...
6    Spain's New Submarine Is Too Big To Fit In Its...
7    US Secretary Of State: "We Want Very Much To H...
8    We Are Nearing The Pain-Point Triggering The "...
9    Philly Fed 'Hope' Plunges To 2 Year Lows As Pr...
dtype: object

In [0]:
from tqdm import tqdm

embeddings = []
for doc in tqdm(nlp.pipe(articles.all_text)):
  embeddings.append(doc.tensor.sum(axis=0))

1001it [07:56, 187.91s/it]

In [0]:
embeddings[0].shape

(768,)